### Importing

In [193]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

air_canada = pd.read_csv('../datasets/new_data.csv')

# seperate dataset into training and predicting
air_canada_to_predict = air_canada[air_canada['choice'] != air_canada['choice']]
air_canada = air_canada[air_canada['choice'] == air_canada['choice']]

X, y = air_canada.drop(['Unnamed: 0', 'id', 'ticket_id', 'choice'], axis=1), air_canada[['choice']]

y['choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

/tmp/ipykernel_224944/2163498253.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })


In [194]:
X.columns

Index(['od', 'origin', 'destination', 'flight_departure_datetime',
       'purchase_datetime', 'time_diff', 'trip_type', 'branded_fare',
       'number_of_pax', 'ADVS_price', 'PREF_price', 'ADVS_price/PREF_price',
       'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory',
       'pref_inv_full', 'advs_ratio', 'pref_ratio', 'total_cap'],
      dtype='object')

array([ 60, 300, 140])

### Preprocessing

In [195]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [207]:
num_pipeline = make_pipeline(
  StandardScaler()
)

cat_pipeline = make_pipeline(
  OneHotEncoder()
)
preprocessing = ColumnTransformer([
    ('cat', cat_pipeline, ['od', 'origin', 'destination', 'trip_type', 'branded_fare', 'total_cap']),
    ('num', num_pipeline, ['time_diff', 'number_of_pax', 'ADVS_price', 'PREF_price', 'ADVS_price/PREF_price',
       'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory', 'pref_inv_full', 'advs_ratio', 'pref_ratio'])
  ], remainder='drop')

In [208]:
X_train_prep = preprocessing.fit_transform(X_train)
X_test_prep = preprocessing.transform(X_test)

In [217]:
X_train_prep[:1]

array([[ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  1.23724248,  1.40794842,  0.5876    ,
         1.66727332, -0.29458418, -1.38287307, -1.12928586, -1.24981903,
         0.57957804,  0.36946862,  0.97580662,  1.40649898]])

### Training

In [210]:
from xgboost import XGBClassifier

bst = XGBClassifier(learning_rate=.3, objective='multi:softmax')
bst.fit(X_train_prep, y_train)

y_pred = bst.predict(X_test_prep)

print("Accuracy:", accuracy_score(y_pred, y_test))
print("F1 Score:", f1_score(y_pred, y_test, average='weighted'))

Accuracy: 0.7465974025974026
F1 Score: 0.7755569730782875


### Tuning

In [212]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],           # Number of boosting rounds
    'learning_rate': [0.01, 0.1, 0.2],         # Step size shrinkage
    'max_depth': [3, 5, 7],                    # Maximum depth of the trees
    'min_child_weight': [1, 3, 5],             # Minimum sum of instance weight needed in a child
    'subsample': [0.6, 0.8, 1.0],              # Subsample ratio of the training instances
    'colsample_bytree': [0.6, 0.8, 1.0],       # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2],                    # Minimum loss reduction to make a further partition on a leaf node
}

rnd_search = GridSearchCV(bst,
    param_grid=param_grid, verbose=3, cv=3, n_jobs=-1,
    scoring='f1')

rnd_search.fit(X_train_prep, y_train)

Fitting 3 folds for each of 2187 candidates, totalling 6561 fits


KeyboardInterrupt: 

### Predicting

In [213]:
X_to_predict, y_to_predict = air_canada_to_predict.drop(['id', 'ticket_id', 'choice'], axis=1), air_canada_to_predict[['id']]
X_to_predict_prep = preprocessing.transform(X_to_predict)
y_to_predict['choice'] = bst.predict(X_to_predict_prep)

/tmp/ipykernel_224944/1900845532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = bst.predict(X_to_predict_prep)


In [214]:
y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})

/tmp/ipykernel_224944/1816271922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})


In [215]:
y_final = pd.concat([y_to_predict, air_canada])[['id', 'choice']]

### Saving

In [216]:
pd.DataFrame.to_csv(y_final, '../predictions/pred7.csv', index=False)